In [1]:
!rm Hawaii.sqlite

In [2]:
#Normal Analytics Dependencies
import matplotlib
import pandas as pd
import numpy as np
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
from sqlalchemy.orm import Session
style.use('seaborn')

#ORM Dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy import create_engine, Column, Integer, String, Float, inspect, MetaData
from sqlalchemy.ext.declarative import declarative_base

# Declaritive base for table schema creation
Base = declarative_base()

In [3]:
# Bringing in the cleaned CSV's
Measurements_path = "Output/Clean_Measurement.csv"
Stations_path = "Output/Clean_Station.csv"
#Reading data
Measurement_df = pd.read_csv(Measurements_path, encoding="utf-8")
Station_df = pd.read_csv(Stations_path, encoding="utf-8")

In [4]:
#Viewing Measurement DF
Measurement_df.head()

,M_ID,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0.00,63
2,3,USC00519397,2010-01-03,0.00,74
3,4,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [5]:
#Viewing Station DF
Station_df.head()

,S_ID,station,name,latitude,longitude,elevation
0,S1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,S2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,S3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,S4,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,S5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [6]:
#Creating a Database workhorse to store both of the cleaned CSV's into a relational tables
engine = create_engine("sqlite:///Hawaii.sqlite")
conn = engine.connect()

In [7]:
 # Define a Measurement class
class Measurement(Base):
    __tablename__ = "Measurement"
    
    M_ID= Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String) 
    prcp = Column(Float) 
    tobs = Column(Integer)

In [8]:
#Defining  a Station Class
class Station(Base):
    __tablename__ = "Station"
    S_ID= Column(String, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    

In [9]:
Base.metadata.create_all(engine)
session = Session(bind=engine)
#session.add(Measurement)
#session.commit()

In [10]:
data_m = Measurement_df.to_dict(orient='records')
data_m[0]

{'M_ID': 1,
 'date': '2010-01-01',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [11]:
data_s = Station_df.to_dict(orient='records')
data_s[0]

{'S_ID': 'S1',
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [12]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [13]:
Measure_Table = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [14]:
Station_Table = sqlalchemy.Table('Station', metadata, autoload=True)

In [15]:
#conn.execute(Measure_Table.delete())

In [16]:
conn.execute(Measure_Table.insert(), data_m)

In [17]:
conn.execute(Station_Table.insert(), data_s)

In [18]:
conn.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [19]:
conn.execute("select * from Station limit 5").fetchall()

[('S1', 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('S2', 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('S3', 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('S4', 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('S5', 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]